In [1]:
import pandas as pd
import plotly.express as px
from IPython.display import display, Math, HTML, Image
import plotly.graph_objects as go
import numpy as np

In [2]:
path = "./DB_12215/"
#read core properties database
file = "Proprietes_ames.csv"
table = pd.read_csv(path + file)

In [3]:

#read csv
file = "Tableau 2.csv"
procedure_generale = pd.read_csv(path + file,delimiter=",")


In [4]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, VBox, Label, HBox

buttonlayout = Layout(overflow_x='hidden',
                    overflow_y='auto',
                      #width='500px',
                    height='29px',
                    #flex_flow='column nowrap',
                    display='flex') 

In [5]:
mldc=np.linspace(1,70000,50)
t=np.linspace(0,24,50)

    
methodeButton = widgets.ToggleButtons(
    options=procedure_generale.loc[6:11,'Sujet'].tolist(),
    description="",
    disabled=False,
    continuous_update=False,
    layout = buttonlayout,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=procedure_generale.loc[6:11,'Article N°'].tolist(),
    style= {'description_width': 'initial'}
#     icons=['check'] * 3
)

lwlButton  = widgets.FloatSlider(
    value=10,
    min=0,
    max=24,
    step=0.01,
    description='Lwl in meters',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    style= {'description_width': 'initial'}
)


mldcButton  = widgets.FloatSlider(
    value=3000,
    min=1,
    max=70000,
    step=10,
    description='mldc in kilogram',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
    style= {'description_width': 'initial'}
)

matiereButton = widgets.ToggleButtons(
    options=table["Reference"].tolist(),
    description="",
    disabled=False,
    continuous_update=False,
    layout = buttonlayout,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=table["Matiere"].tolist(),
    style= {'description_width': 'initial'}
#     icons=['check'] * 3
)

menu1 = HBox([lwlButton,mldcButton]) 
menu2 = VBox([menu1,methodeButton,matiereButton])


In [6]:
def calculate_Bc(lwl,mldc,delta):
    a = 0.084*0.32*delta*lwl
    b = 0.032*delta*lwl**2
    c = -3/25*mldc
    Bc = (-1*b+(b**2-4*a*c)**.5)/2/a
    V = 5*lwl**0.5
    
    return Bc,V

In [7]:
def calculate_kdyn(lwl,mldc,bc,delta,V):
    kdyn = 24**.5/mldc**0.17*2.5
    kdyn = 0.32*(lwl/10/bc+0.084)*delta*V**2*bc**2/mldc
    return kdyn

In [8]:
def tableA7_requirements(methode,lwl,mldc):

    #tau0 contrainte intermédiaire de calcul
    #tau1 contrainte après imposition de la valeur minimale pour la resistance
    #tau2 resistance minimale en cisaillement 2019
    #tau3 resistance minimale en cisaillement 2008
    
    #Tau as a function of lwl 2019 modified by G.Dolto
    tau0 = b-a*lwl
    tau0_max = lwl*0+0.3
    tau1_min = lwl*0+0.58
    tau1 = max(tau0,tau0_max)
    tau2 = min(tau1,tau1_min)
    #Tau as a function of lwl 2008
    if lwl<10:
            tau3 = 0.25
    elif lwl>15:
            tau3 = 0.40
    else:
            tau3 = 0.25+0.03*(lwl-10)
        
    ksls = 1
    bc, V = calculate_Bc(lwl,mldc,38)
    kdyn = 3
    
    
    
    #Sigma as a function of mldc 2019 as required for displacement motor in ABCD
    sigma1 = 0.008 * (2.4*mldc**0.33+20)
    #Sigma as a function of mldc 2019 as required for planning motor in A
    sigma2 = 0.008*0.1*mldc/lwl/bc*(1+kdyn)
    #Sigma as a function of mldc 2019 as required for planning motor in B
    sigma3 = 0.008*0.1*mldc/lwl/bc*(1+0.8**.5*kdyn)
    #Sigma as a function of mldc 2019 as required for planning motor in C
    sigma4 = 0.008*0.1*mldc/lwl/bc*(1+0.6**.5*kdyn)
    #Sigma as a function of mldc 2019 as required for planning motor in C
    sigma5 = 0.008*0.1*mldc/lwl/bc*(1+0.4**.5*kdyn)
    #Sigma as a function of mldc 2019 as required for sailing in ABCD
    sigma6 = 0.008*(2*mldc**.33+18)*ksls
    
    
    if methode == "Méthode 1 (Simplifiée)":
        kAM = 0.90
    elif methode == "Méthode 2 (Développée) analyse pli par pli":
        kAM = 0.95
    else:
        kAM = 1
             

    
    return tau2,tau3,sigma1,sigma2,sigma3,sigma4,sigma5,sigma6,kAM,ksls,kdyn

In [9]:
def update_value(change):
    
    global table
    global valid2019
    global newunvalid
    global taudcomin2019,taudcomin2008,d_abcd_sigmadccomin2019,p_a_sigmadccomin2019,p_b_sigmadccomin2019,p_c_sigmadccomin2019,p_d_sigmadccomin2019,v_abcd_sigmadccomin2019,kAM,ksls,kdyn
    #Numérise
    # a and b from modified

    a = -0.07
    b = -0.12
    taudcomin2019,taudcomin2008,d_abcd_sigmadccomin2019,p_a_sigmadccomin2019,p_b_sigmadccomin2019,p_c_sigmadccomin2019,p_d_sigmadccomin2019,v_abcd_sigmadccomin2019,kAM,ksls,kdyn = tableA7_requirements(methodeButton.value,lwlButton.value,mldcButton.value)
    table["τd"]=[""]*26
    table["σd"]=[""]*26
    for i in range(len(table)):
        try:
            val = float(table.loc[i,"εuf1b"])
            if val <= 0.35:
                table.loc[i,"τd"]=float(table.loc[i,"τ12"])*0.55*.90
                table.loc[i,"σd"]=float(table.loc[i,"σuc1"])*0.55*.90
            elif val > 0.35:
                table.loc[i,"τd"]=float(table.loc[i,"τ12"])*0.65*.90
                table.loc[i,"σd"]=float(table.loc[i,"σuc1"])*0.65*.90
        except ValueError:
            table.loc[i,"τd"]=float(table.loc[i,"τ12"])*0.50*.90
            table.loc[i,"σd"]=float(table.loc[i,"σuc1"])*0.50*.90
    
    valid2019 = table.loc[(table['τd']>taudcomin2019/kAM) & (table['σd']>d_abcd_sigmadccomin2019/kAM),"Reference"].tolist()
    newunvalid1 = table.loc[(table['τd']>taudcomin2008) & (table['τd']<=taudcomin2019/kAM),"Reference"].tolist()
    newunvalid2 = table.loc[(table['τd']>taudcomin2019/kAM) & (table['σd']<=d_abcd_sigmadccomin2019/kAM),"Reference"].tolist()
    newunvalid = newunvalid1 + newunvalid2
    
    shorted = ""
    for i in valid2019:
        shorted = shorted + ", " + i
    
    
    excluded = ""
    for i in newunvalid:
        excluded = excluded + ", " + i
   
    h.value = "2019 compliant :<br>"+shorted+ "<br>"+"2019 excluded :<br>"+excluded
    
    indexchoisi = table[table["Reference"]== matiereButton.value]
    i = indexchoisi.index.values[0]
    
    demonstration = demonstrate(i)
    
    details.value = demonstration
    
    with display.batch_update():
        display.data[4].x = [d_abcd_sigmadccomin2019/kAM,d_abcd_sigmadccomin2019/kAM,4.5,4.5]
        display.data[4].y = [taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM]
        display.data[5].x = [0.2,0.2,4.5,4.5]
        display.data[5].y = [taudcomin2008,1.0,1.0,taudcomin2008]
        display.data[6].x = [p_a_sigmadccomin2019/kAM,p_a_sigmadccomin2019/kAM,4.5,4.5]
        display.data[6].y = [taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM]
        display.data[7].x = [p_b_sigmadccomin2019/kAM,p_b_sigmadccomin2019/kAM,4.5,4.5]
        display.data[7].y = [taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM]
        display.data[8].x = [p_c_sigmadccomin2019/kAM,p_c_sigmadccomin2019/kAM,4.5,4.5]
        display.data[8].y = [taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM]
        display.data[9].x = [p_d_sigmadccomin2019/kAM,p_d_sigmadccomin2019/kAM,4.5,4.5]
        display.data[9].y = [taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM]
        display.data[10].x = [v_abcd_sigmadccomin2019/kAM,v_abcd_sigmadccomin2019/kAM,4.5,4.5]
        display.data[10].y = [taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM]
    

In [25]:
def demonstrate(index,):
    
    #demonstration
    demonstration = "<font size=4>"+table.loc[index,"Reference"]+"</font><br>│"
    demonstration = demonstration + '{:>50} : {:>10}│'.format(
        "Matiere",
        table.loc[index,"Matiere"]
    )
    demonstration = demonstration + '{:>50} : {:>10}kg/m3│'.format(
        "Masse volumique ρ",
        table.loc[index,"Masse volumique ρ"]
    )
    demonstration = demonstration + '{:>10} : {:>10}│'.format(
        "Elongation à la rupture",
        table.loc[index,"εuf1b"]
    )
    demonstration = demonstration + '{:>50} : {:>10}MPa│'.format(
        "Resistance en cisaillement plan, à la rupture",
        table.loc[index,"τ12"]
    )
    demonstration = demonstration + '{:>10} : {:>10}MPa│'.format(
        "Resistance en compression normale, à la rupture",
        table.loc[index,"σuc1"]
    )
    
    
    
    
    demonstration = demonstration + "<br><br>\
    Le matériau <b>\
    "
    demonstration = demonstration + table.loc[index,"Reference"] +"</b> "
    if table.loc[index,"τd"]<taudcomin2008:
        demonstration = demonstration + "\
        n'était pas qualifié \
        "
    else:
        demonstration = demonstration + "\
        était qualifié \
        "
    demonstration = demonstration + "\
        EN-ISO-12215:2008 pour la réalisation de stratifiés sandwich situés en fond de coque \
        "
    demonstration = demonstration + "\
    d'un bateau de <b>\
    "
    demonstration = demonstration + str(lwlButton.value) + 'm</b> '
    demonstration = demonstration + "\
    de longueur de flottaison, et de déplacement total en charge de<b>\
    "
    demonstration = demonstration + str('{:.0f}'.format(mldcButton.value)) + 'kg</b>. '
    demonstration = demonstration + "\
    Depuis le 1er juillet 2021, le matériau \
    "
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>d_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + "\
            conserve sa qualification \
            "
        else:
            demonstration = demonstration + "\
            perds sa qualification \
            "
    else:
        demonstration = demonstration + "\
        n'est pas qualifié non plus \
        "
    demonstration = demonstration + "\
    EN-ISO-12215:2018.<br><br>\
    "   
    
    demonstration = demonstration + "\
    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; D&nbsp;&nbsp;&nbsp;&nbsp; P&nbsp;&nbsp;&nbsp;&nbsp; V&nbsp;&nbsp;&nbsp;&nbsp;<BR>\
    " 
    
    demonstration = demonstration + "\
    <font size=2>Catégorie A&nbsp;&nbsp;&nbsp;&nbsp;</font>\
    "
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>d_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>p_a_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>v_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    demonstration = demonstration + "\
    <br>\
    "
    
    demonstration = demonstration + "\
    <font size=2>Catégorie B&nbsp;&nbsp;&nbsp;&nbsp;</font>\
    "
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>d_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>p_b_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>v_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    demonstration = demonstration + "\
    <br>\
    "
    
    demonstration = demonstration + "\
    <font size=2>Catégorie C&nbsp;&nbsp;&nbsp;&nbsp;</font>\
    "
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>d_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>p_c_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>v_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    demonstration = demonstration + "\
    <br>\
    "
    
    demonstration = demonstration + "\
    <font size=2>Catégorie D&nbsp;&nbsp;&nbsp;&nbsp;</font>\
    "
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>d_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>p_d_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " - "
    
    if table.loc[index,"τd"]>taudcomin2008:
        
        if (table.loc[index,"σd"]>v_abcd_sigmadccomin2019/kAM) & (table.loc[index,"τd"]>taudcomin2019/kAM):
            demonstration = demonstration + " ■&nbsp;&nbsp;&nbsp;&nbsp; "
        else:
            demonstration = demonstration + " □&nbsp;&nbsp;&nbsp;&nbsp; "
    else:
        demonstration = demonstration + " -&nbsp;&nbsp;&nbsp;&nbsp; "
    demonstration = demonstration + "\
    <br>\
    (D) Moteur à déplacement, (P) Moteur plannant, (V) Voilier <br>\
    (■) Qualifié ISO2019, (□) Dé-qualifié ISO2019, (-) Non qualifié ISO2019, ni ISO2008<br>\
    "
    
    
    demonstration = demonstration + "<br>\
    <font size=2>\
    "
    demonstration = demonstration + "\
        ▪ Résistance minimum en cisaillement dans le plan τdco,min = \
    "
    demonstration = demonstration + str('{:.2f}'.format(taudcomin2019)) + "MPa " + "<code> min(max(" + str(b) + "-" + str(a) + " x " + str(lwlButton.value) +  ", 0.3), 0.58)</code><br>"
    demonstration = demonstration + "\
        ▪ Résistance minimum en compression dans le sens normal σdcco,min = \
    "
    demonstration = demonstration + str('{:.2f}'.format(d_abcd_sigmadccomin2019))+ "MPa "  + "<code> 0.008 x PBASE = 0.008 x (2.4 x "+ str('{:.0f}'.format(mldcButton.value))+ "^0.33+20)</code><br>"
    demonstration = demonstration + "\
        ▪ Résistance minimum en cisaillement dans le plan τdco,min2008 = \
    "
    demonstration = demonstration + str('{:.2f}'.format(taudcomin2008))+ "MPa " + "<code> min(max(0.25 + 0.03 x ("+ str(lwlButton.value) +  " - 10), 0.25), 0.40)</code><br>"
    
    try:
        
        val = float(table.loc[index,"εuf1b"])
        if val <= 0.35:
            demonstration = demonstration + "\
            ▪ Contrainte de conception en cisaillement plan τdco = \
            "
            demonstration = demonstration + str('{:.2f}'.format(table.loc[index,"τd"])) + "MPa " + "<code>  " + str(table.loc[index,"εuf1b"]) + "<0.35 " + str(float(table.loc[index,"εuf1b"])<0.35) + ", 0.55 x 0.9 x" + str(table.loc[index,"τ12"]) + "</code>.<br> "
            demonstration = demonstration + "\
            ▪ Contrainte de conception en compression normale σdcco = \
            "
            demonstration = demonstration + str('{:.2f}'.format(table.loc[index,"σd"])) + "MPa " + "<code>  " + str(table.loc[index,"εuf1b"]) + "<0.35 " + str(float(table.loc[index,"εuf1b"])<0.35) + ", 0.55 x 0.9 x" + str(table.loc[index,"σuc1"]) + "</code></font>.<br> "

        elif val > 0.35:
            demonstration = demonstration + "\
            ▪ Contrainte de conception en cisaillement plan τdco = \
            "
            demonstration = demonstration + str('{:.2f}'.format(table.loc[index,"τd"])) + "MPa " + "<code>  " + str(table.loc[index,"εuf1b"]) + "<0.35 " + str(float(table.loc[index,"εuf1b"])<0.35) + ", 0.65 x 0.9 x" + str(table.loc[index,"τ12"]) + "</code>.<br> "
            demonstration = demonstration + "\
            ▪ Contrainte de conception en compression normale σdcco = \
            "
            demonstration = demonstration + str('{:.2f}'.format(table.loc[index,"σd"])) + "MPa " + "<code>  " + str(table.loc[index,"εuf1b"]) + "<0.35 " + str(float(table.loc[index,"εuf1b"])<0.35) + ", 0.65 x 0.9 x" + str(table.loc[index,"σuc1"]) + "</code></font>.<br> "
    except ValueError:
        demonstration = demonstration + "\
        ▪ Contrainte de conception en cisaillement plan τdco = \
        "
        demonstration = demonstration + str('{:.2f}'.format(table.loc[index,"τd"])) + "MPa " + "<code>  " + "balsa core " + str(True) + ", 0.50 x 0.9 x" + str(table.loc[index,"τ12"]) + "</code>.<br> "
        demonstration = demonstration + "\
        ▪ Contrainte de conception en compression normale σdcco = \
        "
        demonstration = demonstration + str('{:.2f}'.format(table.loc[index,"σd"])) + "MPa " + "<code>  " + "balsa core " + str(True) + ", 0.50 x 0.9 x" + str(table.loc[index,"σuc1"]) + "</code></font>.<br> "
   
    
    demonstration = demonstration + "<br>\
    <font size=4>Autres matières</font><br> La qualification EN-ISO-12215:2019 des matières dépends fortement de leur usage final par le constructeur. Ce dernier doit prendre en compte\
    les formes précises du navire et leurs mesures,\
    la méthode d'analyse employée pour le concevoir,\
    ainsi que le domaine d'utilisation pour lequel il est déclaré.\
    Cette qualification des matières doit être contôlée <b>au cas par cas</b> et ce contrôle doit être ré-itéré <b>à chaque mise à jour du produit</b>. Ce contôle peut se conclure par une perte de la qualification de la matière.<br>\
    Par exemple, pour un bateau à moteur à déplacement, de même déplacement et de même longueur que le vôtre, les produits suivants conservent ou perdent leur qualification.\
    "
    
    return demonstration




In [26]:
#init
a = -0.07
b = -0.12
taudcomin2019,taudcomin2008,d_abcd_sigmadccomin2019,p_a_sigmadccomin2019,p_b_sigmadccomin2019,p_c_sigmadccomin2019,p_d_sigmadccomin2019,v_abcd_sigmadccomin2019,kAM,ksls,kdyn = tableA7_requirements(methodeButton.value,lwlButton.value,mldcButton.value)
#print(taudcomin2019,taudcomin2008,d_abcd_sigmadccomin2019,kAM)
table["τd"]=[""]*26
table["σd"]=[""]*26
for i in range(len(table)):
    try:
        val = float(table.loc[i,"εuf1b"])
        if val <= 0.35:
            table.loc[i,"τd"]=float(table.loc[i,"τ12"])*0.55*0.9
            table.loc[i,"σd"]=float(table.loc[i,"σuc1"])*0.55*0.9
        elif val > 0.35:
            table.loc[i,"τd"]=float(table.loc[i,"τ12"])*0.65*0.9
            table.loc[i,"σd"]=float(table.loc[i,"σuc1"])*0.65*0.9
    except ValueError:
        table.loc[i,"τd"]=float(table.loc[i,"τ12"])*0.50*0.9
        table.loc[i,"σd"]=float(table.loc[i,"σuc1"])*0.50*0.9
        
demonstration = demonstrate(0)
details = widgets.HTML(demonstration)


In [27]:
file = open("LOGO2.png", "rb")
image = file.read()
logo = widgets.Image(
    value=image,
    format='png',
    width=111,
    height=111,
)
head_display = HBox([logo,menu2])
head_display

In [28]:
fig = px.scatter(table,
                 x="σd",
                 y="τd", 
                 color="Matiere",
                 text="Reference",
                 #size="Masse volumique ρ",
                 hover_name="Reference",
                 hover_data = ["Masse volumique ρ"])

fig.update_layout(
    title="Ultimate values of the mechanical properties of core materials",
    height = 750,
    font=dict(
        family="Arial",
        size=10,
        #color="RebeccaPurple"
    ),
    legend=dict(
    orientation="h",
    yanchor="top",
    y=-0.05,
    xanchor="left",
    x=0.01
)
)

h=widgets.HTML("")

fig.add_scatter(x=[d_abcd_sigmadccomin2019/kAM,d_abcd_sigmadccomin2019/kAM,4.5,4.5], y=[taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM], fill="toself", name = "EN-ISO-12215:2019 cat.ABCD Displacement motor craft")
fig.add_scatter(x=[0.2,0.2,4.5,4.5], y=[taudcomin2008/kAM,1.0,1.0,taudcomin2008/kAM], fill="toself",name = "EN-ISO-12215:2008 All crafts under 24m")
fig.add_scatter(x=[p_a_sigmadccomin2019/kAM,p_a_sigmadccomin2019/kAM,4.5,4.5], y=[taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM], fill="toself", name = "EN-ISO-12215:2019 cat.A Planning motor craft")
fig.add_scatter(x=[p_b_sigmadccomin2019/kAM,p_b_sigmadccomin2019/kAM,4.5,4.5], y=[taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM], fill="toself", name = "EN-ISO-12215:2019 cat.B Planning motor craft")
fig.add_scatter(x=[p_c_sigmadccomin2019/kAM,p_c_sigmadccomin2019/kAM,4.5,4.5], y=[taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM], fill="toself", name = "EN-ISO-12215:2019 cat.C Planning motor craft")
fig.add_scatter(x=[p_d_sigmadccomin2019/kAM,p_d_sigmadccomin2019/kAM,4.5,4.5], y=[taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM], fill="toself", name = "EN-ISO-12215:2019 cat.D Planning motor craft")
fig.add_scatter(x=[v_abcd_sigmadccomin2019/kAM,v_abcd_sigmadccomin2019/kAM,4.5,4.5], y=[taudcomin2019/kAM,1.0,1.0,taudcomin2019/kAM], fill="toself", name = "EN-ISO-12215:2019 cat.ABCD sailing craft")


display = go.FigureWidget(data = fig)

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    #border='solid',
                    width='100%')

right_layout = Layout(overflow_x='hidden',
                    overflow_y='auto',
                    display='flex',
                    flex_flow='column',
                    height='750px',
                    align_items='stretch',
                    #border='solid',
                    width='100%')

lwlButton.observe(update_value, names='value')
mldcButton.observe(update_value, names='value')
methodeButton.observe(update_value, names='value')
matiereButton.observe(update_value, names='value')
leftdisplay = VBox([display], layout = box_layout)
rightdisplay = VBox([details, h], layout = right_layout)
HBox([leftdisplay,rightdisplay])

    'data': [{'customdata': array([[ 71. ],
                     …